<a href="https://colab.research.google.com/github/neyhartj/BerryBox/blob/master/deploy_BerryBox_FCNSegmentationModel_Colab_20220629.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BerryBox Image Analysis Pipeline

Use this notebook for production applications of a trained fully convolutional network (FCN) to measure quality parameters on individual berries in images.

## Setup

Prior to running the pipeline, make sure you have completed these setup steps:

1. Clone the 

In [ ]:
# Mount 
from google.colab import drive
drive.mount('/content/drive/')
drive = "/content/drive/MyDrive"

!nvidia-smi

Mounted at /content/drive/
Tue Jun 14 15:04:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                             


# **Materials**
  Input the material mask name and information below. Some of the items described here may not appear below, but anything that appears below is described here.

  Specifically:
 
  **name** - The name for the material. This is pretty arbitrary, but it will be
  used to label output folders and images.
 
  **input_rbg_vals** - The rbg values of the material in the input mask image.
 
  **output_val** - The greyscale value of the mask when you output the images.
  This is arbitrary, but every material should have its own output color
  so they can be differentiated
 
  **confidence_threshold** - The lower this number, the more voxels will be labled a specific material. Essentially, the ML algorith outptus a confdience value  (centered on 0.5) for every voxel and every material. By default, voxels with  a confidence of 0.5 or greater are determined to be the material in question.  But we can labled voxles with a lower condience level by changing this  parameter

  **training_image_directory /training_mask_directory**: Input the directory where your training images and masks are located.

  **validation_fraction**: Input the fraction of images you want to validate your model during training. These are not a independent validation, but are part of the training process.

  **num_models**: Enter the number of models you want to iteratively train. Because these are statistical models, the performance of any given model will vary. Training more models will allow you to select the model that best fits your data.
  
  **num_epochs**: Enter number of epochs that you want to use to train your model. More is generally better, but takes more time.

  **batch_size**: Input your batch size. Larger batch sizes allow for faster training, but take up more VRAM. If you are running out of VRAM during training, decrease your batch size.

  **scale**: Input how you want your images scaled during model training and inference. When the scale is 1, your images will be used at full size for training. When the scale is less than 1, your images will be downsized according to the scale you set for training and inference, decreasing VRAM usage. If you run out of VRAM during training, consider rescaling your images.
  
  **normalization_path**: The path to the normalization data file that was saved during model training.

  **models_directory**: Directory where your models are saved.

  **model_group**: Name for the group models you iteratively generate.

  **current_model_name**: Name for each individual model you generate; will automatically be labeled 1 through n for the number of models you specify above.

  **val_images/val_masks**: Input the directory where your independent validation images and masks are located. These images are not used for training and are used as an independent validation of your model.

  **csv_directory**: Directory where a CSV file of your validation results will be saved.

  **inference_directory**: Directory where the images you want analyzed are located.

  **output_directory**: Directory where you want your analysis results to be saved.



In [ ]:
#############################
#### Set user parameters ####
#############################

class Material:
 
  def __init__(self, name, input_rgb_vals, output_val, confidence_threshold=0):
    self.name = name
    self.input_rgb_vals = input_rgb_vals
    self.output_val = output_val
    self.confidence_threshold = confidence_threshold

#Creating a list of materials so we can iterate through it
materials = [
             Material("background", [0,0,0], 0, 0.5),
             Material("berry", [255,255,255], 255, 0.75),
             ]


# What material would you like to make inferences for?
materials_toprint = ["berry"]

# Project directory
# IMPORTANT - ALL DIRECTORIES NEED TO END IN A /
proj_dir =  drive + "/ARS_Cranberry/ImageAnalysis/BerryBox/2021_BerryBox_ImageAnalysis/"

# Distance for the watershed segmentation
distance = 10

# Maximum object area (in pixels) to keep
max_area = 15000
# Minimum object area (in pixels) to keep
min_area = 600

# Properties for regionprops
# region_properties = ["area", "axis_major_length", "axis_minor_length", "eccentricity"] # For local runs
region_properties = ["area", "major_axis_length", "minor_axis_length", "eccentricity"] # For colab runs

# Should the pipeline include watershed segmentation? Note: this can be unreliable
run_watershed = False

# Should the pipeline save segmented images
save_segmentation_image = True

# Input deep learning model path 
# This file should end in ".pth"
model_path = drive + "/ARS_Cranberry/ImageAnalysis/BerryBox/fcn_model_building/model_output/berryBox_fcn_0.0.3/models/berryBox_fcn_0.0.3_model3.pth"

# Normalization data path
normalization_path = drive + "/ARS_Cranberry/ImageAnalysis/BerryBox/fcn_model_building/model_output/berryBox_fcn_0.0.3/berryBox_fcn_0.0.3_normalization_param.txt"

# Directory of images to segment
inference_directory = proj_dir + "/imagesToSegment"



# **Image Segmentation**

Run the image inference pipeline. This pipeline will:
1. Read in an inference image and identify the QR code and scaling
2. Run the image through the prediction model
3. Segment the relevant mask
4. Identify objects in the image
5. Measure object properties and save the results

## Import packages and load a specific model

In [ ]:
# Load relevant packages
import os
import torch
import torch.nn as nn
import numpy as np
import cv2 as cv
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation import fcn_resnet101
import torchvision.transforms as T
from PIL import Image
from scipy import ndimage as ndi
import pandas as pd
from tqdm import tqdm
from skimage.color import rgb2gray, label2rgb
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import feature, segmentation
from skimage.measure import label, regionprops_table, regionprops
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

# Name of the model group
current_model_name = os.path.basename(model_path).replace(".pth", "")
model_group = current_model_name.split("_model")[0]


## Specify directories
# Directory to store output segmented images
output_directory = proj_dir + "/segmentation_output"
# Directory of segmented images
seg_output_directory = output_directory + "/segmented_images"

# Empty and create these directories
for dirname in [output_directory, seg_output_directory]:
    if not os.path.exists(dirname):
        os.mkdir(dirname)

# How many materials?
num_materials = len(materials)



# Load a pretrained model
model = fcn_resnet101(pretrained=False)
model.classifier=FCNHead(2048, num_materials)
device = torch.device('cuda')
model.to(device)
 
# Load the model specified above
model.load_state_dict(torch.load(model_path), strict=False)
model.train()

print("Model loaded!")


## Load the normalization information ##
# Read in the important model training log information
with open(normalization_path, "r") as file:
    for line in file:
        # Strip off newline; separate by tab
        tabs = line.strip().split("\t")

        # Get the name of the variable; this will be used for assignment
        var_name = tabs[0]

        # Parse the second tab
        if tabs[1].startswith("tensor"):

            # Create a vector of numeric characters
            var_value = tabs[1].split("[")[1].split("]")[0].split(", ")
            # Convert this to numeric
            var_value = [float(x) for x in var_value]
            # Convert to np array; then to tensor
            var_value = np.array(var_value)
            var_value = torch.tensor(var_value)

        else:
            var_value = float(tabs[1])

        # Assign variable name
        vars()[var_name] = var_value
        
# assign to mean and std
mean = normalization_mean
std = normalization_std
newW = int(image_scale_newW)
newH = int(image_scale_newH)


# Load a QR code detector
detector = cv.QRCodeDetector()


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

Model loaded!


## Run the image processing pipeline

In [ ]:
## Iterate over images and run through the prediction model ##

# Rename the directory containing the images to segment
dir_name = inference_directory
filenames = os.listdir(dir_name)
print(str(len(filenames)) + " images found.")

# Create an empty list to store region data
all_region_df = []

# Iterate over the images
for i, filename in tqdm(enumerate(filenames)):

# ## TESTING ##
# i = 0
# filename = filenames[i]

    # Open the image
    image = Image.open(dir_name +'/'+ filename)
    image_cv = cv.imread(dir_name + "/" + filename)

    # Rescale the image
    image = image.resize((newW, newH))
    # Convert to gray
    image_gray = np.asarray(image)
    image_gray = rgb2gray(image_gray)
    # image_gray = image_gray[0:newH, 0:newW] # Add this to resize image1
    image = np.array(image, dtype = float)
    new_im = np.zeros((3, newH, newW))
    new_im[0,:,:] = image[:,:,0]
    new_im[1,:,:] = image[:,:,1]
    new_im[2,:,:] = image[:,:,2]
    image_tensor = new_im


    # Create a tensor from the image
    image_tensor = torch.from_numpy(image_tensor)
    # Normalize the tensor and send it to the GPU
    image_tensor = T.Normalize(mean=mean, std=std)(image_tensor)
    image_tensor.unsqueeze_(0)
    image_tensor = image_tensor.to(device=device, dtype=torch.float32)

    # Run the image through the prediction model
    with torch.no_grad():
        mask = model(image_tensor)['out']
        mask = nn.Sigmoid()(mask)
        mask = mask.cpu().detach().numpy()

    # Iterate over materials to print
    for mat_to_print in materials_toprint:
        # Find the index of this material in the materials list
        mat_idx = [i for i, x in enumerate(materials) if x.name == mat_to_print][0]

        # Get the material at this index
        mat = materials[mat_idx]

        # Get the mask from the prediction model at this index
        mat_mask = mask[0,mat_idx,:,:]
        mat_mask[mat_mask >= mat.confidence_threshold] = mat.output_val
        mat_mask[mat_mask < mat.confidence_threshold] = 0

        # Perform object segmentation and regionprop calculation
        # This is from https://github.com/danforthcenter/plantcv/blob/master/plantcv/plantcv/watershed.py
        # Convert the mat_mask to 8-bit
        mat_mask = mat_mask.astype("uint8")

        # Run the watershed if called
        if run_watershed:
            # Run distance transform
            dist_transform = cv.distanceTransformWithLabels(mat_mask, distanceType = cv.DIST_L2, maskSize = 0)[0]
            local_max = feature.peak_local_max(dist_transform, indices = False, min_distance = distance, labels = mat_mask)

            markers = ndi.label(local_max, structure=np.ones((3, 3)))[0]
            dist_transform1 = -dist_transform
            seg1 = segmentation.watershed(dist_transform1, markers, mask = mat_mask)
        else:
            seg1 = mat_mask

        ## Estimate berry traits
        # Label the segmentation output
        label_mat = label(np.array(seg1), background = 0)

        # Regionprops
        region_properties1 = list(set(["label", "bbox"] + region_properties))
        region_properties_names = region_properties1 + [x + "_intensity_mean" for x in ["red", "green", "blue"]] + [x + "_intensity_sd" for x in ["red", "green", "blue"]]
        region_properties_names = region_properties_names + [x + "_intensity_mean" for x in ["hue", "sat", "val"]] + [x + "_intensity_sd" for x in ["hue", "sat", "val"]]
        region_properties_names = tuple(["file_name", "material", "label"] + [x for x in region_properties_names if x != "label"])

        # Empty dictionary to store data
        regions_dict = {}

        # Initialize lists in the dictionary
        for key in region_properties_names:
            regions_dict[key] = []

        # Iterate over regions in the image
        for region in regionprops(label_image = label_mat):

            # Add manual keys
            regions_dict["file_name"] = filename
            regions_dict["material"] = mat_to_print

            # Add props to the dictionary
            for prop in region_properties1:
                regions_dict[prop].append(region[prop])


            # Convert image to HSV
            image_hsv = cv.cvtColor(image.astype("uint8"), cv.COLOR_RGB2HSV)

            berry_rgb_values = []
            berry_hsv_values = []

            for y, x in region.coords:
                berry_rgb_values.append(image[y, x, :])
                berry_hsv_values.append(image_hsv[y, x, :])

            for c, left in enumerate(["red", "green", "blue"]):
                key = left + "_intensity_mean"
                vals = [x[c] for x in berry_rgb_values]
                regions_dict[key].append(np.mean(vals))

                key = key.replace("mean", "sd")
                regions_dict[key].append(np.std(vals))

            for c, left in enumerate(["hue", "sat", "val"]):
                key = left + "_intensity_mean"
                vals = [x[c] for x in berry_hsv_values]
                regions_dict[key].append(np.mean(vals))

                key = key.replace("mean", "sd")
                regions_dict[key].append(np.std(vals))

        # Convert the regions_dict to a data.frame
        regions_df = pd.DataFrame(regions_dict)

        # Remove excessively large regions
        regions_df1 = regions_df[(regions_df["area"] <= max_area) & (regions_df["area"] >= min_area)]
        
        # Save the region data
        all_region_df.append(regions_df1)

        ### Save a segmentation image ###
        if save_segmentation_image:
            image_use = image.astype("uint8")
            image_use = label2rgb(label_mat, image_use, alpha=0.3, bg_label = 0)
            regions_df_use = regions_df1.to_dict()

            # Iterate over the berry index
            for lab in regions_df_use["label"]:
                bbox = regions_df_use["bbox"][lab]
                # draw rectangle around segmented coins
                minr, minc, maxr, maxc = bbox
                cv.rectangle(image_use, (minc, minr), (maxc, maxr),(0,255,0),2)

            image_use_save = Image.fromarray((image_use * 255).astype("uint8"))
            image_use_save.save(seg_output_directory + "/" + mat_to_print + "-segmented-" + filename)
            

# Save the region data
# Merge the region data.feames
all_regions_data = pd.concat(all_region_df)
region_filename = output_directory + "/" + current_model_name + "_InferenceImageRegionData.csv"
all_regions_data.to_csv(region_filename)
            
print("Segmentation complete!")

861 images found.


861it [45:35,  3.18s/it]


Segmentation complete!
